In [1]:
import torch
import torch.nn as nn
import random
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from tqdm import tqdm
import pickle 
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
import numpy as np
import collections 
import cv2
import matplotlib.pyplot as plt
import os

c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
import time

In [3]:
init_size = 50
pop_size = 50
time_remainning = 400
mutate1_prob = 0.05
mutate2_prob = 0.02
mutate3_prob = 0.05
mutate4_prob = 0.05
crossover_prob = 0.1
status = {"small" : 0, "tall" : 1, "fireball" : 2}

In [4]:
class Individual:
    def __init__(self, actions=None, fitness =None):
        self.fitness = 0
        if actions == None:
            actions = []
            for i in range(init_size):
                action = random.randrange(0, len(SIMPLE_MOVEMENT))
                rep = random.randrange(1, 45)
                for j in range(rep):
                    actions.append(action)
            self.actions = actions
            self.evaluate()
        else:
            if fitness == None:
                self.actions = actions
                self.evaluate()
            else:
                self.actions = actions
                self.fitness = fitness

    def evaluate(self):
        state = env.reset()
        fitness = 0
        tmp_actions = []
        info = 0
        cur_status = 0
        for action in self.actions:
            state, reward, done, info = env.step([action])
            if info[0]['flag_get'] == True:
                tmp_actions.append(action)
                fitness += reward[0]
                break
            if  'terminal_observation' in info[0] or status[info[0]['status']] < cur_status:
                for i in range(90):
                    if len(tmp_actions) > 1:
                        del tmp_actions[-1]
                self.actions = tmp_actions
                self.evaluate()
                return
            else:
                cur_status = status[info[0]['status']]
                tmp_actions.append(action)
                fitness += reward[0]
        self.actions = tmp_actions
        self.fitness = fitness + info[0]['score'] * 5

In [5]:
class Population:
    def __init__(self, save_file = "", gen = 0):
        self.pop = []
        self.gen = 0
        if(save_file == ""):
            for i in range(0, pop_size):
                print(i)
                self.pop.append(Individual())
        else:
            self.loadFileToPop(save_file, gen)

    def evolve(self):
        for ind in self.pop:
            print(ind.fitness)
        print("--------------------------------------------")
        length = len(self.pop)
        for i in range(0, length - pop_size):
            del self.pop[-1]
        a = random.randrange(0, int(pop_size / 10))
        while True:
            b = random.randrange(0, int(pop_size / 5))
            if b != a:
                break
        self.pop[a] = self.mutate2(self.pop[a])
        self.pop[a] = self.mutate3(self.pop[a])
        self.pop[b] = self.mutate2(self.pop[b])
        self.pop[b] = self.mutate3(self.pop[b])
        for i, ind in enumerate(self.pop):
            alpha = random.random()
            if alpha <= mutate1_prob:
                self.pop[i] = self.mutate1(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob:
                self.pop[i] = self.mutate2(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob + mutate3_prob:
                self.pop[i] = self.mutate3(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob + mutate3_prob + mutate4_prob:
                self.pop[i] = self.mutate4(self.pop[i])    
        for i in range(int(crossover_prob * pop_size)):
            a = random.randrange(0, int(pop_size / 5))
            while True:
                b = random.randrange(0, pop_size)
                if b != a:
                    break
            ind1, ind2 = self.crossOver(copy.deepcopy(self.pop[a]), copy.deepcopy(self.pop[b]))
            self.pop.append(ind1)
            self.pop.append(ind2)
        self.pop.sort(key=lambda x: x.fitness, reverse=True)
        self.gen += 1

    def crossOver(self, ind1, ind2):
        child1 = []
        child2 = []
        cut_ind = random.randrange(0, min(len(ind1.actions), len(ind2.actions)))
        for i in range(0, cut_ind):
            child1.append(ind1.actions[i])
            child2.append(ind2.actions[i])
        child1.append(int((ind1.actions[cut_ind] + ind2.actions[cut_ind]) / 2))
        child2.append(int((ind1.actions[cut_ind] + ind2.actions[cut_ind]) / 2))
        for i in range(cut_ind + 1, len(ind2.actions)):
            child1.append(ind2.actions[i])
        for i in range(cut_ind + 1, len(ind1.actions)):
            child2.append(ind1.actions[i])
        return Individual(child1), Individual(child2)

    # chen vao giua chuoi hanh dong
    def mutate1(self, ind):
        t = copy.deepcopy(ind)
        start = random.randrange(0, len(t.actions))
        action = random.randrange(0, len(SIMPLE_MOVEMENT))
        rep = random.randrange(1, 45)
        actions = t.actions[:start] + [action for i in range(rep)] + t.actions[start:]
        x = Individual(actions)
        del t
        if x.fitness > ind.fitness:
            return x
        else:
            return ind

    # chen vao cuoi chuoi hanh dong
    def mutate2(self, ind):
        t = copy.deepcopy(ind)
        res = []
        res2 = []
        rep = random.randrange(1, 45)
        for action in range(1, len(SIMPLE_MOVEMENT) - 1):
            if(action == 2):
                continue
            if(action == 4 and (ind.actions[-1] == 4 or ind.actions[-1] == 2)):
                actions = t.actions + [0] + [action for i in range(rep)]
            else:
                actions = t.actions + [action for i in range(rep)]
            x = Individual(actions)
            print(action, x.fitness)
            if x.fitness > t.fitness:
                res.append(x)
            if x.fitness == t.fitness:
                res2.append(x)
        if len(res) == 0:
            if(len(res2) == 0):
                for i in range(90):
                    if len(t.actions) > 0:
                        del t.actions[-1]
                t.evaluate()
                return t
            else:
                return res2[random.randrange(0, len(res2))]
        return res[random.randrange(0, len(res))]
            
    # chen nhay vao cuoi chuoi hanh dong
    def mutate3(self, ind):
        t = copy.deepcopy(ind)
        rep = random.randrange(15, 45)
        actions = t.actions + [0]
        actions = actions + [4 for i in range(rep)]
        x = Individual(actions)
        del t
        if x.fitness > ind.fitness:
            return x
        else:
            return ind   
            
    # chen chuoi vao giua
    def mutate4(self, ind):
        t = copy.deepcopy(ind)
        rep = random.randrange(1, 45)
        action = random.randrange(0, len(SIMPLE_MOVEMENT))
        start = random.randrange(0, len(t.actions))
        actions = t.actions
        for i in range(start, min(len(actions), start + rep)):
            actions[i] = action
        x = Individual(actions)
        if x.fitness > ind.fitness:
            return x
        else:
            return ind
    
    def savePopToFile(self, save_file):
        save = open(save_file, 'w')
        for ind in self.pop:
            for j in ind.actions:
                save.write(str(j) + ' ')
            save.write(str(ind.fitness) + '\n')
        save.close()
    
    def loadFileToPop(self, save_file, gen):
        self.gen = gen
        self.pop = []
        save = open(save_file, 'r')
        lines = save.readlines()
        for line in lines:
            tmp = tuple(map(float, line.split(" ")[:]))
            actions = []
            for i in range(0, len(tmp) - 1):
                actions.append(int(tmp[i]))
            fitness = tmp[-1]
            self.pop.append(Individual(actions, fitness))
        save.close()


In [6]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 240 * 256 * 3:
            img = np.reshape(frame, [240, 256, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]),
                                                dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


def make_env(env):
    env = MaxAndSkipEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    env = ScaledFloatFrame(env)
    return JoypadSpace(env, SIMPLE_MOVEMENT)

In [7]:
class DQNSolver(nn.Module):

    def __init__(self, input_shape, n_actions):
        super(DQNSolver, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)
    

In [8]:
class DQNAgent:

    def __init__(self, state_space, action_space, max_memory_size, batch_size, gamma, lr,
                 dropout, exploration_max, exploration_min, exploration_decay, double_dq, pretrained):

        self.state_space = state_space
        self.action_space = action_space
        self.double_dq = double_dq
        self.pretrained = pretrained
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if self.double_dq:  
            self.local_net = DQNSolver(state_space, action_space).to(self.device)
            self.target_net = DQNSolver(state_space, action_space).to(self.device)
            
            if self.pretrained:
                self.local_net.load_state_dict(torch.load(mario_rl + "dq1.pt", map_location=torch.device(self.device)))
                self.target_net.load_state_dict(torch.load(mario_rl + "dq2.pt", map_location=torch.device(self.device)))
                    
            self.optimizer = torch.optim.Adam(self.local_net.parameters(), lr=lr)
            self.copy = 5000  
            self.step = 0
        else:  
            self.dqn = DQNSolver(state_space, action_space).to(self.device)
            
            if self.pretrained:
                self.dqn.load_state_dict(torch.load(mario_rl + "dq.pt", map_location=torch.device(self.device)))
            self.optimizer = torch.optim.Adam(self.dqn.parameters(), lr=lr)

        self.max_memory_size = max_memory_size
        if self.pretrained:
            self.STATE_MEM = torch.load(mario_rl + "STATE_MEM.pt")
            self.ACTION_MEM = torch.load(mario_rl + "ACTION_MEM.pt")
            self.REWARD_MEM = torch.load(mario_rl + "REWARD_MEM.pt")
            self.STATE2_MEM = torch.load(mario_rl + "STATE2_MEM.pt")
            self.DONE_MEM = torch.load(mario_rl + "DONE_MEM.pt")
            with open(mario_rl + "ending_position.pkl", 'rb') as f:
                self.ending_position = pickle.load(f)
            with open(mario_rl + "num_in_queue.pkl", 'rb') as f:
                self.num_in_queue = pickle.load(f)
        else:
            self.STATE_MEM = torch.zeros(max_memory_size, *self.state_space)
            self.ACTION_MEM = torch.zeros(max_memory_size, 1)
            self.REWARD_MEM = torch.zeros(max_memory_size, 1)
            self.STATE2_MEM = torch.zeros(max_memory_size, *self.state_space)
            self.DONE_MEM = torch.zeros(max_memory_size, 1)
            self.ending_position = 0
            self.num_in_queue = 0
        
        self.memory_sample_size = batch_size
        
        self.gamma = gamma
        self.l1 = nn.SmoothL1Loss().to(self.device) # Also known as Huber loss
        self.exploration_max = exploration_max
        self.exploration_rate = exploration_max
        self.exploration_min = exploration_min
        self.exploration_decay = exploration_decay

    def remember(self, state, action, reward, state2, done):
        self.STATE_MEM[self.ending_position] = state.float()
        self.ACTION_MEM[self.ending_position] = action.float()
        self.REWARD_MEM[self.ending_position] = reward.float()
        self.STATE2_MEM[self.ending_position] = state2.float()
        self.DONE_MEM[self.ending_position] = done.float()
        self.ending_position = (self.ending_position + 1) % self.max_memory_size  # FIFO tensor
        self.num_in_queue = min(self.num_in_queue + 1, self.max_memory_size)
        
    def recall(self):
        idx = random.choices(range(self.num_in_queue), k=self.memory_sample_size)
        
        STATE = self.STATE_MEM[idx]
        ACTION = self.ACTION_MEM[idx]
        REWARD = self.REWARD_MEM[idx]
        STATE2 = self.STATE2_MEM[idx]
        DONE = self.DONE_MEM[idx]
        
        return STATE, ACTION, REWARD, STATE2, DONE

    def act(self, state):
        # Epsilon-greedy 
        
        if self.double_dq:
            self.step += 1
        if random.random() < self.exploration_rate:  
            return torch.tensor([[random.randrange(self.action_space)]])
        if self.double_dq:
            return torch.argmax(self.local_net(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()
        else:
            return torch.argmax(self.dqn(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()

    def copy_model(self):
        
        self.target_net.load_state_dict(self.local_net.state_dict())
    
    def experience_replay(self):
        
        if self.double_dq and self.step % self.copy == 0:
            self.copy_model()

        if self.memory_sample_size > self.num_in_queue:
            return

        STATE, ACTION, REWARD, STATE2, DONE = self.recall()
        STATE = STATE.to(self.device)
        ACTION = ACTION.to(self.device)
        REWARD = REWARD.to(self.device)
        STATE2 = STATE2.to(self.device)
        DONE = DONE.to(self.device)
        
        self.optimizer.zero_grad()
        if self.double_dq:
            #  Q*(S, A) <- r + γ max_a Q_target(S', a)
            target = REWARD + torch.mul((self.gamma * 
                                        self.target_net(STATE2).max(1).values.unsqueeze(1)), 
                                        1 - DONE)

            current = self.local_net(STATE).gather(1, ACTION.long()) 
        else:
            # Q*(S, A) <- r + γ max_a Q(S', a) 
            target = REWARD + torch.mul((self.gamma * 
                                        self.dqn(STATE2).max(1).values.unsqueeze(1)), 
                                        1 - DONE)
                
            current = self.dqn(STATE).gather(1, ACTION.long())
        
        loss = self.l1(current, target)
        loss.backward() # Compute gradients
        self.optimizer.step() # Backpropagate error

        self.exploration_rate *= self.exploration_decay
        
        self.exploration_rate = max(self.exploration_rate, self.exploration_min)

In [9]:
# cài tđặt thư mục chứa model của RL
mario_rl = 'RL_save_model/'
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = make_env(env)  # Wraps the environment so that frames are grayscale 
observation_space = env.observation_space.shape
action_space = env.action_space.n   
pretrained = True
agent = DQNAgent(state_space=observation_space,
                     action_space=action_space,
                     max_memory_size=30000,
                     batch_size=32,
                     gamma=0.90,
                     lr=0.00025,
                     dropout=0.,
                     exploration_max=0.001,
                     exploration_min=0.001,
                     exploration_decay=0.99,
                     double_dq=True,
                     pretrained=pretrained)

In [10]:
# Chọn map
name_map = '1-1'
env = gym_super_mario_bros.make('SuperMarioBros-' + name_map + '-v0')
env = make_env(env)  # Wraps the environment so that frames are grayscale 
observation_space = env.observation_space.shape
action_space = env.action_space.n   
env.reset()
num_episodes = 10
for ep_num in tqdm(range(num_episodes)):
    state = env.reset()
    state = torch.Tensor([state])
    while True:
        action = agent.act(state)
        state_next, reward, terminal, info = env.step(int(action[0]))
        time.sleep(0.1)
        env.render()
        state_next = torch.Tensor([state_next])
        reward = torch.tensor([reward]).unsqueeze(0)
        terminal = torch.tensor([int(terminal)]).unsqueeze(0)
        state = state_next
        if  terminal:
            break
    num_episodes += 1 

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 50%|█████████████████████████████████████████▌                                         | 5/10 [01:27<01:27, 17.41s/it]


KeyboardInterrupt: 